## UCL COMP0029 Individual Project for Year 3 BSc
### Robust Robotic Grasping Utilising Touch Sensing - Proposed Learning Framework Notebook
This notebook contains the essential code for the proposed Multilayer Perceptron (MLP) approach to grasp stability prediction. The whole notebook should take approximately 10 minutes to run.

### 1. Load packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import gc

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.decomposition import PCA

import seaborn as sns

Set device for `PyTorch` training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

PyTorch configs

In [ ]:
gc.collect()
torch.cuda.empty_cache()
torch.set_num_threads(2)

In [ ]:
np.errstate(invalid='ignore', divide='ignore')

### 2. Load datasets from saved .npy files

To collect data for this experiment, you can run the "Collect Sensory Data" button in the Pybullet simulation. This generates a predefined number of Gaussian grasps randomly generated from a base hand pose. Each individual grasp is considered as an individual experiment, and the data collected from this experiment is split into four, each stored in its own dataset.

For all object models used in this experiment, each object has 4 datasets which include:
- `depth_ds.npy` which stores the depth tactile data from the mounted DIGIT sensors
- `color_ds.npy` which stores the colored (RGB) version of the depth tactile data from the mounted DIGIT sensors
- `poses_ds.npy` which stores the randomly-generated 6d hand poses from the simulation
- `outcomes_ds.npy` which stores the outcomes of each random pose

In this notebook, we will use the following dataset combinations:
- `tactile` only (depth + colour)
- `visual` without geometric features
- `visual` with geometric features
- `multi-modal` consisting of `tactile` and `visual`
- `complete` consisting of `tactile` and `visual` with geometric features

In [ ]:
root = "../../datasets/"

In [ ]:
object_names = ["block1", "block2", "block3", "cylinder1", "cylinder2", "cylinder3", "bottle1", "bottle2", "bottle3"]

Load `.npy` datasets via `np.load()`

In [ ]:
depth_data = np.empty((0, 2, 160, 120))
color_data = np.empty((0, 2, 160, 120, 3))
poses_data = np.empty((0, 6))
grasp_outcomes_data = np.empty((0,))

for object_name in object_names:
    # Construct the relative paths of each dataset and load them into the notebook
    depth_data_temp = np.load(root + object_name + "_ds/depth_ds.npy", mmap_mode='r')
    colour_data_temp = np.load(root + object_name + "_ds/color_ds.npy", mmap_mode='r')
    poses_data_temp = np.load(root + object_name + "_ds/poses_ds.npy", mmap_mode='r')
    grasp_labels_temp = np.load(root + object_name + "_ds/grasp_outcomes.npy", mmap_mode='r')

    depth_data = np.append(depth_data, depth_data_temp, axis=0)
    color_data = np.append(color_data, colour_data_temp, axis=0)
    poses_data = np.append(poses_data, poses_data_temp, axis=0)
    grasp_outcomes_data = np.append(grasp_outcomes_data, grasp_labels_temp, axis=0)

In [ ]:
del depth_data_temp, colour_data_temp, poses_data_temp, grasp_labels_temp
torch.cuda.empty_cache()

These datasets should all be in the form of $(N\times...)$ where $N$ is the number of examples:

In [ ]:
print(f"Shape of depth_data: {depth_data.shape}")
print(f"Shape of color_data: {color_data.shape}")
print(f"Shape of poses_data: {poses_data.shape}")
print(f"Shape of grasp_outcomes_data: {grasp_outcomes_data.shape}")

Additionally, we confirm the number of successful and unsuccessful grasps recorded. This helps us in the next section to determine how many examples we should include for each class in order to produce a balanced dataset.

In [ ]:
print(f"# of sucessesful grasps: {(grasp_outcomes_data == 1).sum()}")
print(f"# of unsuccessful grasps: {(grasp_outcomes_data == 0).sum()}")

### 3. Preprocessing

In this section, we create the required datasets specified from section 2, and normalise the datasets.

#### 3.1 Normalization

In [ ]:
def normalize(arr):
    # Normalize & standardize each column
    mean = np.mean(arr, axis=0)
    std = np.std(arr, axis=0)
    
    arr = (arr - mean) / std
    arr = np.nan_to_num(arr, 0)
    arr[np.isinf(arr)] = 0
    return arr

Depth and colour data consists of pairs of tactile sensory readings. Thus, we concatenate each pair together into single images

In [ ]:
depth_data = np.concatenate((depth_data[:, 0], depth_data[:, 1]), axis=2)
color_data = np.concatenate((color_data[:, 0], color_data[:, 1]), axis=2)

In [ ]:
depth_data = torch.from_numpy(normalize(depth_data))
color_data = torch.from_numpy(normalize(color_data))
visual_data = torch.from_numpy(np.nan_to_num(normalize(poses_data)))

In [ ]:
print(f"Shape of depth_ds: {depth_data.shape}")
print(f"Shape of colour_ds: {color_data.shape}")
print(f"Shape of visual_ds: {visual_data.shape}")

Create the tactile dataset

In [ ]:
tactile_data = torch.cat([depth_data.unsqueeze(-1), color_data], dim=-1)
tactile_data = torch.nan_to_num(tactile_data)

In [ ]:
print(f"Shape of tactile_ds: {tactile_data.shape}")

In [ ]:
del depth_data, color_data, poses_data
torch.cuda.empty_cache()

#### Dimensionality reduction

In [ ]:
# A simple convolutional neural network that extracts features from an input tensor
class FeatureExtractorCNN(nn.Module):
    def __init__(self):
        super(FeatureExtractorCNN, self).__init__()
        self.conv1 = nn.Conv2d(4, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=3)


    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        x = self.conv4(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        return x

In [ ]:
# Preprocess data using CNN feature extraction
cnn = FeatureExtractorCNN()
cnn_tactile = torch.cat([cnn(img.float().permute(2,0,1)).unsqueeze(0) for img in tactile_data])
cnn_tactile = cnn_tactile.reshape(cnn_tactile.shape[0], -1)
cnn_tactile.shape

In [ ]:
del tactile_data

### 4. Testing different data representations

In our MLP model, there are two fully connected (dense) layers, each with an activation function (ReLU for the first layer and no activation for the second layer). The input size, hidden size, and output size are parameters that need to be specified when creating an instance of the MLP.

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = optim.Adam(self.parameters())

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x
        
    def train_mlp(self, epochs, X_train, y_train):
        self.losses = []
        for i in range(epochs):
            inputs = torch.from_numpy(X_train).float()
            labels = torch.from_numpy(y_train).float().view(-1, 1)

            # Zero the gradients
            self.optimizer.zero_grad()

            # Forward pass
            preds = self(inputs)
            loss = self.criterion(preds, labels)
            self.losses.append(loss.item())

            # Backward pass and optimization
            loss.backward()
            self.optimizer.step()

            if i % 10 == 0:
                torch.cuda.empty_cache()

    def eval_mlp(self, X_test, y_test, verbose=False):
        # Evaluate the performance of the model on the testing set
        with torch.no_grad():
            inputs = torch.from_numpy(X_test).float()
            labels = torch.from_numpy(y_test).float().view(-1, 1)

            # Forward pass
            final_preds = self(inputs)
            predicted = (final_preds > 0).float()
            
            # Confusion matrix
            if verbose:
                cm = confusion_matrix(y_test, predicted)
                sns.heatmap(cm, linewidths=1, annot=True, fmt='g')

            # Performance metrics
            accuracy = round(accuracy_score(labels, predicted) * 100, 2)
            precision = round(precision_score(labels, predicted) * 100, 2)
            recall = round(recall_score(labels, predicted) * 100, 2)
            f1 = round(f1_score(labels, predicted) * 100, 2)
            return accuracy, precision, recall, f1
    
    def plot_losses(self):
        plt.plot(self.losses)
        plt.xlabel('Epochs')
        plt.ylabel('Training Loss')
        plt.show()

#### 4a. Testing datatset combinations (CNN dimensionality-reduced) without geometric features

In [ ]:
def train_mlp_multitrial(dataset, grasp_outcomes_data, trials_count: int = 5, sampling=False):
    performance_metrics = np.empty((0, 4))
    dataset = dataset.detach().numpy() if isinstance(dataset, torch.Tensor) else dataset

    for i in range(trials_count):
        X_train, X_test, y_train, y_test = train_test_split(dataset, grasp_outcomes_data, test_size=0.2)
        mlp = MLP(input_size=X_train.shape[1], hidden_size=64, output_size=1)
        mlp.train_mlp(epochs=500, X_train=X_train, y_train=y_train)
        accuracy, precision, recall, f1 = mlp.eval_mlp(X_test=X_test, y_test=y_test)
        metrics_row = np.array([accuracy, precision, recall, f1]).reshape(1, 4)
        performance_metrics = np.append(performance_metrics, metrics_row, axis=0)

        torch.cuda.empty_cache()
    return performance_metrics

Train Tactile-only dataset + CNN

In [ ]:
metrics = train_mlp_multitrial(cnn_tactile, grasp_outcomes_data, trials_count=5)

In [ ]:
print("Tactile-only MLP")
print(f"Accuracy:{metrics[:, 0]}")
print(f"Mean accuracy score: {np.mean(metrics[:, 0]):.2f} + {np.std(metrics[:, 0]):.2f}")
print(f"Mean precision score: {np.mean(metrics[:, 1]):.2f}  + {np.std(metrics[:, 1]):.2f}")
print(f"Mean recall score: {np.mean(metrics[:, 2]):.2f}  + {np.std(metrics[:, 2]):.2f}")
print(f"Mean f1 score: {np.mean(metrics[:, 3]):.2f} + {np.std(metrics[:, 3]):.2f}")

In [ ]:
torch.cuda.empty_cache()

Train Visual-only dataset + CNN

In [ ]:
metrics = train_mlp_multitrial(visual_data, grasp_outcomes_data, trials_count=5)

In [ ]:
print("Visual-only MLP")
print(f"Accuracy:{metrics[:, 0]}")
print(f"Mean accuracy score: {np.mean(metrics[:, 0]):.2f} + {np.std(metrics[:, 0]):.2f}")
print(f"Mean precision score: {np.mean(metrics[:, 1]):.2f}  + {np.std(metrics[:, 1]):.2f}")
print(f"Mean recall score: {np.mean(metrics[:, 2]):.2f}  + {np.std(metrics[:, 2]):.2f}")
print(f"Mean f1 score: {np.mean(metrics[:, 3]):.2f} + {np.std(metrics[:, 3]):.2f}")

In [ ]:
torch.cuda.empty_cache()

Train Multi-modal dataset (tactile + visual) + CNN

In [ ]:
# We simply combine the cnn-processed tactile data (from Section 5.3.1) with the visual data
cnn_complete = torch.cat([cnn_tactile.reshape(cnn_tactile.shape[0], -1), visual_data], dim=1)

In [ ]:
metrics = train_mlp_multitrial(cnn_complete, grasp_outcomes_data, trials_count=5)

In [ ]:
print("Multi-modal MLP")
print(f"Accuracy:{metrics[:, 0]}")
print(f"Mean accuracy score: {np.mean(metrics[:, 0]):.2f} + {np.std(metrics[:, 0]):.2f}")
print(f"Mean precision score: {np.mean(metrics[:, 1]):.2f}  + {np.std(metrics[:, 1]):.2f}")
print(f"Mean recall score: {np.mean(metrics[:, 2]):.2f}  + {np.std(metrics[:, 2]):.2f}")
print(f"Mean f1 score: {np.mean(metrics[:, 3]):.2f} + {np.std(metrics[:, 3]):.2f}")

In [ ]:
del cnn_complete
torch.cuda.empty_cache()

### 5. Determining impact of geometric features of objects on MLP accuracy
This section aims to determine the capability of the selected object features (principal components) in identifying primitive object classes.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import StandardScaler


np.set_printoptions(precision=2, formatter={'float_kind': "{:.3f}".format})

#### 5a. Clustering for object features using PCA

Loading object features

In [ ]:
### Object features
block_features = np.array([
    [0.025, 0.05, 0.05, 0., 0., 0., 0., 0., 0.],
    [0.03, 0.025, 0.045, 0., 0., 0., 0., 0., 0.],
    [0.05, 0.025, 0.04, 0., 0., 0., 0., 0., 0.],
])

cylinder_features = np.array([
    [0.04, 0.04, 0.05, 950.21606561, 14540.28434464, 950.21606561, 14540.28434464, 950.21606561, 14540.28434464],
    [0.045, 0.045, 0.035, 750.78800246, 11488.6197291, 750.78800246, 11488.6197291, 750.78800246, 11488.6197291],
    [0.034, 0.034, 0.045, 1315.17794549, 20124.96103064, 1315.17794549, 20124.96103064, 1315.17794549, 20124.96103064]
])

bottle_features = np.array([
    [0.06, 0.04, 0.04, 43195.64459266, 114198.0441697 , 45229.93706864, 135651.61794731, 75768.06626518,  83802.00991944],
    [0.04, 0.06, 0.06, 68993.34322089, 220902.86884084, 75354.47923164, 239160.99530455, 109695.41304924, 147938.06047763],
    [0.04, 0.06, 0.04, 61744.75459905, 168925.8805044 , 68143.84372052, 148775.07141178, 73102.67367022, 102953.40831915]
])

object_names = {0: 'cylinder', 1: 'block', 2: 'bottle'}

Feature transformation - concatenating features together

In [ ]:
object_features = (block_features, cylinder_features, bottle_features)
features = np.concatenate(object_features)
labels = np.array([0, 0, 0, 1, 1, 1, 2, 2, 2])

Standardize the data

In [ ]:
scaler = StandardScaler()
features = scaler.fit_transform(features)

PCA

In [ ]:
def pca(data, k, verbose=True):
    pca = PCA(n_components=k)
    pca.fit(data)

    if verbose:
        print(f"Variance captured: {sum(pca.explained_variance_ratio_)*100:.2f}%")
    return pca.transform(data)

Determine the number of components to use

In [ ]:
for i in range(1, 5):
    pca_res = pca(features, k=i)

In [ ]:
### Plot 3-feature PCA results
pca3d = pca(features, k=3)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

colors = ['r', 'g', 'b']
for i, c in zip(np.unique(labels), colors):
    ax.scatter(pca3d[labels == i, 0], pca3d[labels == i, 1], pca3d[labels == i, 2], c=c, label=object_names[i])

ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
ax.set_zlabel('Component 3')
ax.set_aspect('equal', 'box')
ax.legend()
plt.show()

#### 5b. MLP: Dataset combination 4: Visual (EE pose and geometric features) + CNN

In [ ]:
blocks = np.repeat(np.concatenate(object_features), 160, axis=0)

visual_data = np.concatenate((visual_data, blocks), axis=1)
visual_data = torch.from_numpy(np.nan_to_num(normalize(visual_data)))
visual_data.shape

In [ ]:
del object_features, blocks

Train Visual-only (EE pose + geometric features) + CNN

In [ ]:
metrics = train_mlp_multitrial(visual_data, grasp_outcomes_data, trials_count=5)

In [ ]:
print("Visual data (EE pose + geo) MLP")
print(f"Accuracy:{metrics[:, 0]}")
print(f"Mean accuracy score: {np.mean(metrics[:, 0]):.2f} + {np.std(metrics[:, 0]):.2f}")
print(f"Mean precision score: {np.mean(metrics[:, 1]):.2f}  + {np.std(metrics[:, 1]):.2f}")
print(f"Mean recall score: {np.mean(metrics[:, 2]):.2f}  + {np.std(metrics[:, 2]):.2f}")
print(f"Mean f1 score: {np.mean(metrics[:, 3]):.2f} + {np.std(metrics[:, 3]):.2f}")

#### 5c. MLP: Dataset combination 5: Tactile + Visual (EE pose and geometric features) + CNN

In [ ]:
torch.cuda.empty_cache()

In [ ]:
final_dataset = torch.cat((cnn_tactile, visual_data), dim=1)

In [ ]:
metrics = train_mlp_multitrial(final_dataset, grasp_outcomes_data, trials_count=5)

In [ ]:
print(f"Final dataset (no PCA) MLP")
print(f"Accuracy:{metrics[:, 0]}")
print(f"Mean accuracy score: {np.mean(metrics[:, 0]):.2f} + {np.std(metrics[:, 0]):.2f}")
print(f"Mean precision score: {np.mean(metrics[:, 1]):.2f}  + {np.std(metrics[:, 1]):.2f}")
print(f"Mean recall score: {np.mean(metrics[:, 2]):.2f}  + {np.std(metrics[:, 2]):.2f}")
print(f"Mean f1 score: {np.mean(metrics[:, 3]):.2f} + {np.std(metrics[:, 3]):.2f}")

In [ ]:
del cnn_tactile

#### 5d. MLP: Selecting number of components for PCA (with dataset combination 5)

We conduct the following experiment to determine the optimal value of $k$ for PCA:
1. Train the MLP from $k=1$ to $k=20$
2. Repeat 1 for 5 times to find the average accuracy for each $k$, and choose $k$ value for the highest mean accuracy

In [ ]:
MIN_K, MAX_K = 1, 30

In [ ]:
pca_accuracies_mean = []
pca_accuracies_std = []
for i in range(MIN_K, MAX_K+1):
    transformed_dataset = pca(final_dataset.detach().numpy(), k=i, verbose=False)
    transformed_dataset = torch.from_numpy(transformed_dataset)
    metrics = train_mlp_multitrial(transformed_dataset, grasp_outcomes_data)
    pca_accuracies_mean.append(np.mean(metrics[:, 0]))
    pca_accuracies_std.append(np.std(metrics[:, 0]))
    torch.cuda.empty_cache()
    print(f"Completed PCA training for k={i} components")

In [ ]:
del transformed_dataset

In [ ]:
del visual_data

Calculate the average accuracies for each $k$

In [ ]:
pca_accuracies_mean = np.array(pca_accuracies_mean)
pca_accuracies_std = np.array(pca_accuracies_std)

In [ ]:
K = pca_accuracies_mean.argmax(axis=0)
print(f"Number of components for PCA: {K}")

In [ ]:
x = np.arange(MIN_K, MAX_K+1)
plt.errorbar(x, pca_accuracies_mean, yerr=pca_accuracies_std, fmt='o', markersize=5, ecolor='red', capsize=3, capthick=1)
plt.grid(True)
plt.xlabel("Number of principal components")
plt.xticks(np.arange(MIN_K, MAX_K+1, step=2))
plt.ylabel("Mean MLP accuracy (%)")
plt.title("Impact of Number of Principal Components on MLP Accuracy")
plt.show()

In [ ]:
x = np.arange(MIN_K, MAX_K+1, step=1)
plt.plot(x, pca_accuracies_mean)
plt.fill_between(x, pca_accuracies_mean-pca_accuracies_std, pca_accuracies_mean+pca_accuracies_std, alpha=0.2)
plt.grid(True)
plt.xlabel("Number of principal components")
plt.xticks(np.arange(MIN_K, MAX_K+1, step=2))
plt.ylabel("Mean MLP accuracy (%)")
plt.title("Impact of Number of Principal Components on MLP Accuracy")
plt.show()

In [ ]:
top_accuracy_indices = np.argsort(pca_accuracies_mean)[-5:] 
top_accuracies = pca_accuracies_mean[top_accuracy_indices]
top_stds = pca_accuracies_std[top_accuracy_indices]

print("Top 5 mean accuracies:", top_accuracies)
print("Top 5 std accuracies:", top_stds)
print("Corresponding indices:", top_accuracy_indices + 1)

### 6. Experiments

We update the train_mlp_multitrial function to accomodate pre-defined training and testing datasets

In [ ]:
def train_mlp_multitrial2(X_train, X_test, y_train, y_test, trials_count: int = 5):
    performance_metrics = np.empty((0, 4))

    for i in range(trials_count):
        mlp = MLP(input_size=X_train.shape[1], hidden_size=64, output_size=1)
        mlp.train_mlp(epochs=500, X_train=X_train, y_train=y_train)
        accuracy, precision, recall, f1 = mlp.eval_mlp(X_test=X_test, y_test=y_test)
        metrics_row = np.array([accuracy, precision, recall, f1]).reshape(1, 4)
        performance_metrics = np.append(performance_metrics, metrics_row, axis=0)

        torch.cuda.empty_cache()
    
    return performance_metrics

#### 6.1 Train MLP on 2 blocks, test on remaining block
Objective: see MLP's robustness

In [ ]:
combinations = [
    [[1, 2], [3]],
    [[1, 3], [2]],
    [[2, 3], [1]]
]

exp1_accuracies_mean = []
exp1_accuracies_std = []

In [ ]:
final_dataset = final_dataset.detach().numpy()

for i in range(3):  # For each primitive object type (box, cylinder, bottle)
    stidx = 480 * i # Starting index to slice final dataset
    for combination in combinations:
        training, testing = combination[0], combination[1]
        X_train1 = final_dataset[(stidx + (training[0] - 1) * 160 + 1):(stidx + (training[0] * 160))]
        X_train2 = final_dataset[(stidx + (training[1] - 1) * 160 + 1):(stidx + (training[1] * 160))]
        X_train = np.vstack((X_train1, X_train2))

        y_train1 = grasp_outcomes_data[(stidx + (training[0] - 1) * 160 + 1):(stidx + (training[0] * 160))]
        y_train2 = grasp_outcomes_data[(stidx + (training[1] - 1) * 160 + 1):(stidx + (training[1] * 160))]
        y_train = np.concatenate((y_train1, y_train2))
        

        X_test = final_dataset[(stidx + (testing[0] - 1) * 160 + 1):(stidx + (testing[0] * 160))]
        y_test = grasp_outcomes_data[stidx + (testing[0] - 1) * 160 + 1:stidx + (testing[0] * 160)]
        metrics = train_mlp_multitrial2(X_train, X_test, y_train, y_test)
        exp1_accuracies_mean.append(np.mean(metrics[:, 0]))
        exp1_accuracies_std.append(np.std(metrics[:, 0]))
        torch.cuda.empty_cache()

In [ ]:
del X_train, X_test, y_train, y_test, training, testing, combination, combinations

In [ ]:
exp1_accuracies_mean = np.array(exp1_accuracies_mean)
exp1_accuracies_std = np.array(exp1_accuracies_std)

In [ ]:
exp1_accuracies_mean

In [ ]:
exp1_accuracies_std

#### 6.2 Train MLP on randomly sampled subset of all objects
Objective: see influence of dataset size on accuracy;

Randomly sample a dataset of the specified size without shuffling the dataset.

In [ ]:
def consistent_sampling(dataset1, dataset2, input_size):
    num_rows = len(dataset1)
    indices = np.random.choice(len(dataset1), input_size, replace=False)
    samples1, samples2 = [], []
    for i in indices:
        samples1.append(dataset1[i])
        samples2.append(dataset2[i])
    
    samples1 = np.stack(samples1, axis=0)
    samples2 = np.stack(samples2, axis=0)
    return samples1, samples2

In [ ]:
exp2_accuracies_mean = []
exp2_accuracies_std = []

for input_size in range(100, 1500, 100):
    dataset, grasp_labels = consistent_sampling(final_dataset, grasp_outcomes_data, input_size=input_size)
    metrics = train_mlp_multitrial(dataset, grasp_labels)
    exp2_accuracies_mean.append(np.mean(metrics[:, 0]))
    exp2_accuracies_std.append(np.std(metrics[:, 0]))

In [ ]:
exp2_accuracies_mean = np.array(exp2_accuracies_mean)
exp2_accuracies_std = np.array(exp2_accuracies_std)

In [ ]:
x = np.arange(100, 1500, step=100)
plt.plot(x, exp2_accuracies_mean)
plt.fill_between(x, exp2_accuracies_mean-exp2_accuracies_std, exp2_accuracies_mean+exp2_accuracies_std, alpha=0.2)
plt.grid(True)
plt.xlabel("Sample size")
plt.xticks(np.arange(100, 1500, step=100), rotation=45)
plt.ylabel("Mean MLP Accuracy (%)")
plt.title("Impact of Sample Size on MLP Performance")
plt.show()